# Chapter 8  Tree-Based Methods

- [Lab 8.3.1 Fitting Classification Trees](#lab-8.3.1)
- [Lab 8.3.2 Fitting Regression Trees](#lab-8.3.2)
- [Lab 8.3.3 Bagging and Random Forests](#lab-8.3.3)
- [Lab 8.3.4 Boosting](#lab-8.3.4)

### Imports and Configurations

In [1]:
# Use rpy2 for loading R datasets
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import data as rdata
from rpy2.robjects import pandas2ri

# Math and data processing
import numpy as np
import scipy as sp
import pandas as pd

# StatsModels
import statsmodels.api as sm
import statsmodels.formula.api as smf

# scikit-learn
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report

# Visulization
from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.style.use('ggplot')

<a id='lab-8.3.1'></a>

### Lab 8.3.1 Fitting Classification Trees

<a id='lab-8.3.2'></a>

### Lab 8.3.2 Fitting Regression Trees

<a id='lab-8.3.3'></a>

### Lab 8.3.3 Bagging and Random Forests

<a id='lab-8.3.4'></a>

### Lab 8.3.4 Boosting